# Getting all available Swiss petrol stations using the overpass Web API

## Libraries and settings

In [1]:
# Libraries
import os
import json
import folium
import requests
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

/workspaces/spatial_data_analysis/Python_Basic_GIS_Functionality/Challenge


## Overpass turbo query to get available petrol stations

In [2]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["amenity"="fuel"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

# Save data to .json file
with open('petrol_stations.json', 'w') as json_file:
    json.dump(data, json_file)

# Store data in data frame
df = pd.json_normalize(data)
df.head()

,type,id,lat,lon,tags.amenity,tags.fixme,tags.name,tags.operator,tags.brand,tags.fuel:diesel,...,tags.fuel:Shell_V-Power,tags.payment:post_finance,tags.covered,tags.payment:girocard,tags.payment:vpay,tags.fuel:diesel_plus,tags.construction,tags.dog,tags.payment:hubrol_card,tags.payment:noten_euro_reka-pay_postcard_mastercard_avia-karte_visa_ec_karte_american_express_reka-karte_myone
0,node,734402,46.690599,7.668189,fuel,Please check name and brand. Agip becomes Eni....,Agip,Agip,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,1228781,46.878598,7.542326,fuel,NaN,NaN,BP,BP,yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,5702030,46.680546,7.827412,fuel,NaN,Migrol,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,5702071,46.683415,7.844559,fuel,NaN,Avia,Avia,Avia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,8082505,46.683817,7.664427,fuel,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Save data frame to a .csv file

In [3]:
# Write data frame to .csv - file
df.to_csv('petrol-stations.csv', index=False, sep=";", encoding='utf-8')

## Use a pivot table to count the number of petrol stations per operator

In [4]:
# Create pivot table
tab = pd.pivot_table(df[['id', 'tags.name']], 
                       values=['id'], 
                       index=['tags.name'],
                       aggfunc='count')

# Sort pivot table
tab_sorted = tab.sort_values(by=['id'], ascending=False)
tab_sorted

,id
tags.name,
Avia,157
Agrola,143
BP,109
Tamoil,95
Shell,76
...,...
Garage de la Prairie,1
Garage de la Station,1
Garage des 3 Sapins,1


## Plot petrol stations on map

In [5]:
# Subset of petrol stations by brand
locations = df[["lat", 
                "lon", 
                "tags.brand", 
                "tags.shop"]].loc[df["tags.brand"].isin(['Migrol', 
                                                         'Agip', 
                                                         'Shell'])]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                   popup=location_info["tags.brand"]).add_to(map)

# Plot map
map

          lat       lon tags.brand tags.shop
9   47.686119  8.621418      Shell       NaN
20  47.495696  8.712981      Shell       NaN
24  47.381593  8.509545      Shell       NaN
29  47.381110  8.492148     Migrol       NaN
33  47.337432  8.530242      Shell       NaN


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [6]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.2.0-1015-azure
Datetime: 2023-11-10 16:04:24
Python Version: 3.10.13
-----------------------------------
